In [1]:
puzzle_input = """####.
.##..
##.#.
###..
##..#"""

In [2]:
from collections import namedtuple

def part_1_solution(raw_input):
    grid = parse_grid(raw_input)
    
    sim = simulate(grid.height, grid.width)
    seen_grids = set()
    
    while grid not in seen_grids:
        seen_grids.add(grid)
        grid = process_one_minute(grid, sim)            
    
    return grid.biodiversity_rating()

BUG = '#'
EMPTY = '-'
    
class Grid:
    def __init__(self, height, width):
        self.height = height
        self.width = width
        self.rows = []
        for _ in range(self.height):
            self.rows.append([EMPTY] * self.width)

    def __str__(self):
        output = ''
        for row in self.rows:
            for cell in row:
                output += cell
            output += '\n'
        return output
    
    def __hash__(self):
        flattened = list()
        for row in self.rows:
            for obj in row:
                flattened.append(obj)
        
        return hash("".join(flattened))
    
    def __eq__(self, other):
        return self.rows == other.rows
    
    def biodiversity_rating(self):
        rating = 0
        i = 0
        for row in self.rows:
            for obj in row:
                if obj == BUG:
                    rating += 2**i
                i += 1
                
        return rating

    def query(self, y, x):
        if y >= self.height or x >= self.width:
            return EMPTY
        
        if y < 0 or x < 0:
            return EMPTY
        
        return self.rows[y][x]

    def assign(self, y, x, state):
        self.rows[y][x] = state


def parse_grid(raw_input):
    grid = Grid(5, 5)
    
    for y, row in enumerate(raw_input.split("\n")):
        for x, obj in enumerate(row):
            if obj == "#":
                grid.assign(y, x, BUG)
                
    return grid


def process_one_minute(grid, sim):
    new_grid = Grid(grid.height, grid.width)
    item = next(sim)
    while item is not TICK:
        if isinstance(item, Query):
            state = grid.query(item.y, item.x)
            item = sim.send(state)
        else:  
            new_grid.assign(item.y, item.x, item.state)
            item = next(sim)
    return new_grid
    
TICK = object()

def simulate(height, width):
    while True:
        for y in range(height):
            for x in range(width):
                yield from step_cell(y, x)
        yield TICK

        
Query = namedtuple('Query', ('y', 'x'))

def count_neighbors(y, x):
    n_ = yield Query(y + 1, x + 0)  
    e_ = yield Query(y + 0, x + 1)
    s_ = yield Query(y - 1, x + 0)
    w_ = yield Query(y + 0, x - 1)
    
    neighbor_states = [n_, e_, s_, w_]
    count = 0
    for state in neighbor_states:
        if state == BUG:
            count += 1
            
    return count

Transition = namedtuple('Transition', ('y', 'x', 'state'))

def step_cell(y, x):
    state = yield Query(y, x)
    neighbors = yield from count_neighbors(y, x)
    next_state = game_logic(state, neighbors)
    yield Transition(y, x, next_state)
    

def game_logic(state, neighbors):
    if state == BUG:
        if neighbors == 1:
            return BUG
        else:
            return EMPTY
    
    if neighbors == 1 or neighbors == 2:
        return BUG
    
    return EMPTY


In [3]:
test_input = """....#
#..#.
#..##
..#..
#...."""
assert(part_1_solution(test_input) == 2129920)
print("Test passed")

Test passed


In [4]:
print(f"Part 1 solution: {part_1_solution(puzzle_input)}")

Part 1 solution: 18400821


In [79]:
from collections import namedtuple, defaultdict

def part_2_solution(raw_input, minutes_to_simulate):
    initial_grid = parse_grid(raw_input)
    
    sim = simulate(5, 5)
    grids = defaultdict(lambda: Grid(5, 5))
    grids[0] = initial_grid
    
    for i in range(minutes_to_simulate):
        grids = process_one_minute(grids, sim)            

    cnt = 0
    for lvl, grid in grids.items():
        cnt += grid.bugs_cnt()
        
    return cnt

BUG = '#'
EMPTY = '-'
RECURSIVE_BLOCK = (2, 2)

class Grid:
    def __init__(self, height, width):
        self.height = height
        self.width = width
        self.rows = []
        for _ in range(self.height):
            self.rows.append([EMPTY] * self.width)

    def __str__(self):
        output = ''
        for row in self.rows:
            for cell in row:
                output += cell
            output += '\n'
        return output
    
    def bugs_cnt(self):
        bugs_cnt = 0
        for y, row in enumerate(self.rows):
            for x, obj in enumerate(row):
                if (x, y) == RECURSIVE_BLOCK:
                    continue
                    
                if obj == BUG:
                    bugs_cnt += 1
                
        return bugs_cnt

    def query(self, y, x, current_lvl, all_grids):
        if self.is_block_outside(y, x):
            if y >= self.height:
                parent_x = RECURSIVE_BLOCK[0]
                parent_y = RECURSIVE_BLOCK[1] + 1
            elif x >= self.width:
                parent_x = RECURSIVE_BLOCK[0] + 1
                parent_y = RECURSIVE_BLOCK[1]
            elif y < 0:
                parent_x = RECURSIVE_BLOCK[0]
                parent_y = RECURSIVE_BLOCK[1] - 1
            elif x < 0:
                parent_x = RECURSIVE_BLOCK[0] - 1
                parent_y = RECURSIVE_BLOCK[1]
                
            parent_lvl = current_lvl - 1
            parent_grid = all_grids[current_lvl - 1]

            return parent_grid.query(parent_y, parent_x, parent_lvl, all_grids)
        
        if (x, y) == RECURSIVE_BLOCK:
            pass
        
        return self.rows[y][x]

    def assign(self, y, x, state):
        self.rows[y][x] = state
        
    def is_block_outside(self, y, x):
        if y >= self.height or x >= self.width:
            return True
        
        if y < 0 or x < 0:
            return True
        
        return False


def parse_grid(raw_input):
    grid = Grid(5, 5)
    
    for y, row in enumerate(raw_input.split("\n")):
        for x, obj in enumerate(row):
            if obj == "#":
                grid.assign(y, x, BUG)
                
    return grid


def process_one_minute(grids, sim):
    new_grids = defaultdict(lambda: Grid(5,5))
    grids_to_visit = sorted(grids.keys())
    
    first_lvl = grids_to_visit[0]
    
    if grids[first_lvl].bugs_cnt() != 0:
        grids_to_visit.insert(0, first_lvl - 1)
        
    last_lvl = grids_to_visit[-1]
    
    if grids[last_lvl].bugs_cnt() != 0:
        grids_to_visit.append(last_lvl + 1)
    
    for current_lvl in grids_to_visit:
        new_grid = Grid(5, 5)

        item = next(sim)
        while item is not GRID_COMPLETED:
            if item is Level:
                item = sim.send(current_lvl) 
            elif isinstance(item, Query):
                grid = grids[item.lvl]
                state = grid.query(item.y, item.x, item.lvl, grids)
                item = sim.send(state)
            elif isinstance(item, Transition):  
                new_grid.assign(item.y, item.x, item.state)
                item = next(sim)
            else:
                raise Exception(f"Wrong item: {item}")
                
        new_grids[current_lvl] = new_grid
            
    return new_grids
    
GRID_COMPLETED = object()

def simulate(height, width):
    while True:
        for y in range(height):
            for x in range(width):
                if (x, y) == RECURSIVE_BLOCK:
                    continue

                yield from step_cell(y, x)
        yield GRID_COMPLETED

        
Query = namedtuple('Query', ('y', 'x', 'lvl'))

def count_neighbors(y, x, current_lvl):
    n_ = Query(y + 1, x + 0, current_lvl)  
    e_ = Query(y + 0, x + 1, current_lvl)
    s_ = Query(y - 1, x + 0, current_lvl)
    w_ = Query(y + 0, x - 1, current_lvl)
    
    count = 0
    for neighbor in [n_, e_, s_, w_]:
        if (neighbor.x, neighbor.y) == RECURSIVE_BLOCK:
            if x == RECURSIVE_BLOCK[0] + 1 and y == RECURSIVE_BLOCK[1]: 
                child_neighbors = [(4, 0), (4, 1), (4, 2), (4, 3), (4, 4)]
            elif x == RECURSIVE_BLOCK[0] - 1 and y == RECURSIVE_BLOCK[1]: 
                child_neighbors = [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4)]
            elif x == RECURSIVE_BLOCK[0] and y == RECURSIVE_BLOCK[1] + 1: 
                child_neighbors = [(0, 4), (1, 4), (2, 4), (3, 4), (4, 4)]
            elif x == RECURSIVE_BLOCK[0] and y == RECURSIVE_BLOCK[1] - 1: 
                child_neighbors = [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0)]
            else:
                raise Exception("Wrong state")
                
            for child_x, child_y in child_neighbors:
                child_state = yield Query(child_y, child_x, current_lvl + 1)
                if child_state == BUG:
                    count += 1
        else:
            neighbor_state = yield neighbor
            if neighbor_state == BUG:
                count += 1        
    return count

Transition = namedtuple('Transition', ('y', 'x', 'lvl', 'state'))
Level = object()

def step_cell(y, x):
    curent_lvl = yield Level
    state = yield Query(y, x, curent_lvl)
    neighbors = yield from count_neighbors(y, x, curent_lvl)
    next_state = game_logic(state, neighbors)
    yield Transition(y, x, curent_lvl, next_state)
    

def game_logic(state, neighbors):
    if state == BUG:
        if neighbors == 1:
            return BUG
        else:
            return EMPTY
    
    if neighbors == 1 or neighbors == 2:
        return BUG
    
    return EMPTY


In [80]:
test_input = """....#
#..#.
#.?##
..#..
#...."""
assert(part_2_solution(test_input, 10) == 99)
print("Test passed")

Test passed


In [81]:
print(f"Part 2 solution: {part_2_solution(puzzle_input, 200)}")

Part 2 solution: 1914
